# Introduction to NLP Fundamentals in TensorFlow

NLP is focused on deriving information from natural language (text or speech).

Another common term for NLP problems is sequence to sequence (seq2seq)

In [1]:
## Check for GPU
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-000cd691-3ea0-4d29-9ce1-2ce917dbe716)


In [2]:
# Import functions from helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

2023-01-25 14:55:52.236483: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Get a text dataset

I am using the the [Kaggle's Introduction to NLP dataset](https://www.kaggle.com/competitions/nlp-getting-started), a set of tweets for binary classifications of disasters.

I will be downloading the dataset from [my course github here](https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip). 

In [3]:
# downloads the dataset from google storage
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-01-25 14:56:47--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.115.128, 172.253.122.128, 142.251.16.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.115.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.2’

100%[======================================>] 607,343     --.-K/s   in 0.01s   

2023-01-25 14:56:47 (45.2 MB/s) - ‘nlp_getting_started.zip.2’ saved [607343/607343]



In [4]:
# Unzip the data
unzip_data("nlp_getting_started.zip")

## Visualizing a text dataset

Need to visualize our text dataset using Python's Pandas.

**NOTE**: A Pandas dataframe can only be the same size as RAM.

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# check out test dataframe 
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# quantity of examples of each class
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

> This is fairly balanced, no balancing required. Otherwise look up imbalanced classification

In [9]:
# Total samples
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Visualize random training samples
import random
random_index = random.randint(0, len(train_df) - 5) # create random indexes not higher than total samples
for row in train_df_shuffled[["text", "target"]][random_index: random_index + 5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not a real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 0 (not a real disaster)
Text:
If you find your patio table umbrella and chairs flipped over and suspect foul play (instead of windstorm) you may be a suspense writer.

---

Target: 0 (not a real disaster)
Text:
Oh wait I expected to go a totally different route in LS that was derailed by another barely passing grade in a required course. Super.

---

Target: 1 (real disaster)
Text:
The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/RlPTtkBG4W

---

Target: 1 (real disaster)
Text:
Motorcyclist bicyclist injured in Denver collision on Broadway: At least two people were taken to a localÛ_ http://t.co/ozK1QHJVfh

---

Target: 1 (real disaster)
Text:
Oregon's biggest wildfire slows growth http://t.co/P0GoS5URXG via @katunews

---



### Split data into training and validation data set

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# use train_test_split to split training data into train and validation splits
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                           train_df_shuffled["target"].to_numpy(),
                                                                           test_size=0.1,
                                                                           random_state=42)

In [13]:
# check the lengths of each split
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
# check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## converting text into numbers

There are multiple ways to convert text to numbers:
* Tokenization - direct mapping of token (word or character) to a number
* Embedding - Creates a vector for each word of arbitrary length (128, 256, 512, etc) and creates a matrix to support showing relationships. The embedding matrix acts as a layer that can be trained as more text is analyzed.

### Text vectorization (tokenization)

In [15]:
# Look at the data
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

# use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in our vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation", # lowercase and removes punctuation
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None, # pads each sequence to be the same length as the longest sequence is
                                    pad_to_max_tokens=False)

In [17]:
# find the average number of tokens (words) in the training tweets
total_words = sum([len(i.split()) for i in train_sentences])

# this produces a float so we round to get an int
average_words = round(total_words / len(train_sentences))

average_words

15

In [18]:
# set up text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be, condenses sentences to this max length

text_vectorizer = TextVectorization(max_tokens=max_vocab_length, # max vocab of 10k words
                                   output_mode="int",
                                   output_sequence_length=max_length) # truncate or pad to match the maximum sequence length of 15 in our case

In [19]:
# Fit the text
text_vectorizer.adapt(train_sentences)

In [20]:
# create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
# choose random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original Text:\n {random_sentence}\n\n Vectorized Version:")
text_vectorizer([random_sentence])

Original Text:
 quick shut down the show take the stage down evacuate everyone from mthe premises Louis is upset

 Vectorized Version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1787, 1476,  134,    2,  431,  167,    2, 1469,  134,  279,  238,
          20,    1, 9610, 1193]])>

In [22]:
# find the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in training data
top_5_words = words_in_vocab[:5] # get most common words
bottom_5_words = words_in_vocab[-5:]

print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding Layer

To make the embedding I will be using the TensorFlow Embedding Layer via `tf.keras.layers.Embedding`

Params for Embedding Layer:
* `input_dim` - Size of the vocabulary
* `output_dim` - the size of the output embedding vector, the vector representation of the word will be this many float values long
* `input_length` - length of sequences being passed to the embedding layer (set to 15 earlier)

In [23]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128,
                             input_length=max_length) # how long is each input

In [24]:
# get random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original Text:\n {random_sentence}\n\nEmbedded Version:\n")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text:
 Emergency responders prepare for chemical disaster through Hazmat training. http://t.co/q9zixCi8E6

Embedded Version:



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04044591, -0.02900395,  0.00997239, ...,  0.03864953,
         -0.00851812, -0.0404544 ],
        [ 0.02330944,  0.04993999,  0.02765414, ..., -0.01541353,
         -0.03966758, -0.01031746],
        [-0.04903242, -0.0224415 ,  0.04273274, ...,  0.02645764,
         -0.03135502, -0.03429221],
        ...,
        [-0.03267145, -0.03186846,  0.01067818, ..., -0.03734125,
          0.03614335, -0.03921503],
        [-0.03267145, -0.03186846,  0.01067818, ..., -0.03734125,
          0.03614335, -0.03921503],
        [-0.03267145, -0.03186846,  0.01067818, ..., -0.03734125,
          0.03614335, -0.03921503]]], dtype=float32)>

> 3 dimensions: 1 sequence, 15 tokens, 128 float values representing each token (word).

In [25]:
# check out single token's embedding
sample_embed[0][0], sample_embed[0][0].shape,  random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.04044591, -0.02900395,  0.00997239, -0.00868595, -0.00701711,
        -0.02441148,  0.02870316, -0.00010179, -0.02191069, -0.03122919,
        -0.00932238, -0.02784938,  0.02494702,  0.03708695,  0.02900923,
         0.01676026,  0.04741515,  0.03222844, -0.04998943,  0.00816886,
        -0.04940176, -0.03884759,  0.04566295, -0.03069859, -0.02465651,
        -0.01160507, -0.04732863,  0.03235627,  0.03983065, -0.03149676,
         0.03988333, -0.04337195, -0.01219127,  0.0164099 , -0.03149553,
         0.01813369,  0.01193737, -0.00101575,  0.02224777, -0.01937242,
        -0.03313792, -0.00638754,  0.03997408,  0.02434206,  0.04397711,
         0.00078727, -0.03038983, -0.04513228, -0.02476666, -0.03299035,
        -0.03510851,  0.00858132,  0.04969701,  0.01831826,  0.00352351,
        -0.00194596, -0.0178342 , -0.00087573, -0.02530595, -0.00017934,
         0.03657316,  0.02232499, -0.0454228 , -0.04936112, -0.00162574,
  

## modeling a text dataset (series of experiments)

The modeling experiements will be:

* Model 0: Naive Bays (baseline, sci-kit learn)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM Model (RNN)
* Model 3: GRU Model (RNN)
* Model 4: Bidirectional-LSTM model (RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of the training data (to explore a data constrained scenario)

Steps for modeling:
1. Create a model
2. Build a model
3. Fit a model
4. Evaluate the model

### Model 0: Baseline model

To build the baseline, I'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.

> It seems commonplace to use non-DL machine learning models to establish a baseline only moving on to Deep Learning when ready to improve.

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# create tokenization and modeling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
    ("clf", MultinomialNB()) # model the text
])

# fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
# evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Baseline achieves accuracy of: {baseline_score * 100:.2f}%")

Baseline achieves accuracy of: 79.27%


In [28]:
# Make prediction
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### creating an evaluation function for model experiments

Using the following metrics:
* Accuracy
* Precision
* Recall
* F1-Score

In [29]:
# Fucntion to evaluate: accuracy, precision, recall, F1-Score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_preds):
    """
    Calculates model accuracy, precision, recall, and f1 score for a binary classification model.
    """
    # calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_preds) * 100
    # calculate model precision, recall, and f1-score 'weighted' average (supports label imbalance)
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_preds, average="weighted")
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1": model_f1}
    
    return model_results

In [30]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_preds=baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## model 1: a simple dense model

In [31]:
# create a tensorboard callback (create new one for each model)
from helper_functions import create_tensorboard_callback

# create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [32]:
# build a model with the funcitonal api
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # one dimensional and of type string only
x = text_vectorizer(inputs) # turn input text into numbers
x = embedding(x) # create an embedding of the numeric inputs
x = layers.GlobalAveragePooling1D(name="global_avg_pool_layer")(x) # condence feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, binary outputs = 1 and activation of sigmoid
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [33]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_avg_pool_layer (Glob  (None, 128)              0         
 alAveragePooling1D)                                             
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [34]:
# compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [35]:
# fit the model
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                    experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230125-145713
Epoch 1/5
215/215 [==============================] - 15s 5ms/step - loss: 0.6088 - accuracy: 0.6885 - val_loss: 0.5317 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4405 - accuracy: 0.8196 - val_loss: 0.4680 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3455 - accuracy: 0.8621 - val_loss: 0.4559 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2831 - accuracy: 0.8934 - val_loss: 0.4662 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2362 - accuracy: 0.9142 - val_loss: 0.4833 - val_accuracy: 0.7848


In [36]:
# check results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4833 - accuracy: 0.7848


[0.4832990765571594, 0.7847769260406494]

In [37]:
# make predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [38]:
# check out the probability matrix
model_1_pred_probs[:5]

array([[0.3363727 ],
       [0.8122002 ],
       [0.9970092 ],
       [0.12435926],
       [0.09801292]], dtype=float32)

In [39]:
# convert model prediction probabilities to model format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [40]:
# calculate model_1 results
model_1_results = calculate_results(val_labels,
                                    model_1_preds)

model_1_results

{'accuracy': 78.4776902887139,
 'precision': 0.7904782347685264,
 'recall': 0.7847769028871391,
 'f1': 0.7814477888250547}

In [41]:
import numpy as np

# check all values of model_1 against baseline
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

> We see that only the F1 score resulted in a higher value over the baseline

## Visualizing learned embeddings

In [42]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [43]:
# model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_avg_pool_layer (Glob  (None, 128)              0         
 alAveragePooling1D)                                             
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [44]:
# get the weight matrix of embedding layer (numerical representation of each token in our training data)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape) # same size as vocab and embedding_dim (output dim of our embedding layer)

(10000, 128)


Tensorflow has a tool called projector: http://projector.tensorflow.org/

Tensorflow embeddings documentation: https://www.tensorflow.org/text/guide/word_embeddings

In [45]:
# create embedding files, this code is in embedding documentation
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

> Manually downloaded the `metadata.tsv` and `vectors.tsv`

## Recurrent Neural Network

RNN's are primarily used for sequence data.

RNN's use the representation of a previous input for a later representation.

## Model 2: LSTM

LSTM = Long Short-Term Memory

Architecture:
```
Input (text) -> Tokenize -> Embedding -> Layers (RNN/Dense) -> Output (label probability)
```

In [46]:
# This kernel is representing the shape incorrectly for LSTM and GRU, this modifies the expected shape of the data
tf.keras.backend.set_image_data_format("channels_last")

# create an LSTM model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.LSTM(units=64, return_sequences=True)(x) # when stacking RNN cells together, need to return_sequences
x = layers.LSTM(64)(x)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [47]:
# get a summary of model_2
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [48]:
# compile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [49]:
# Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230125-145734
Epoch 1/5
215/215 [==============================] - 31s 8ms/step - loss: 0.2196 - accuracy: 0.9178 - val_loss: 0.5329 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1559 - accuracy: 0.9413 - val_loss: 0.5833 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1275 - accuracy: 0.9505 - val_loss: 0.6541 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1064 - accuracy: 0.9606 - val_loss: 0.7135 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0908 - accuracy: 0.9651 - val_loss: 0.8451 - val_accuracy: 0.7717


In [50]:
# make predictions with LSTM model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[1.8630134e-02],
       [8.9566916e-01],
       [9.9969900e-01],
       [1.0713756e-01],
       [4.7146552e-04],
       [9.9744833e-01],
       [6.3294834e-01],
       [9.9974746e-01],
       [9.9957126e-01],
       [4.6334100e-01]], dtype=float32)

In [51]:
# convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [52]:
# calculate model 2 results
model_2_results = calculate_results(y_true=val_labels, y_preds=model_2_preds)
model_2_results

{'accuracy': 77.16535433070865,
 'precision': 0.7738402637184817,
 'recall': 0.7716535433070866,
 'f1': 0.7693184984034235}

In [53]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [54]:
# Create a GRU model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation="tanh")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [55]:
# get model 3 summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [56]:
# compile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [57]:
# fit model 3
model_3.fit(train_sentences,
            train_labels,
            validation_data=(val_sentences, val_labels),
            epochs=5,
            callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                   "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230125-145811
Epoch 1/5
215/215 [==============================] - 4s 8ms/step - loss: 0.3326 - accuracy: 0.9145 - val_loss: 1.5927 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1695 - accuracy: 0.9585 - val_loss: 1.2300 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1509 - accuracy: 0.9698 - val_loss: 1.7005 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1406 - accuracy: 0.9739 - val_loss: 1.7896 - val_accuracy: 0.7717
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1539 - accuracy: 0.9707 - val_loss: 1.5222 - val_accuracy: 0.7651


In [58]:
# make predictions with GRU model
model_3_pred_probs = model_3.predict(val_sentences)

24/24 [==============================] - 0s 3ms/step


In [59]:
# convert model 3 pred probs to labels
model_3_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [60]:
# calculate model 3 results
model_3_results = calculate_results(y_true=val_labels, y_preds=model_3_preds)
model_3_results

{'accuracy': 77.16535433070865,
 'precision': 0.7738402637184817,
 'recall': 0.7716535433070866,
 'f1': 0.7693184984034235}

In [61]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 4: Bidirectional RNN

In [62]:
# Create a bidirectional RNN
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidirectional")

In [63]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [64]:
# compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [65]:
# fit data
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230125-145821
Epoch 1/5
215/215 [==============================] - 6s 12ms/step - loss: 0.1209 - accuracy: 0.9626 - val_loss: 0.7757 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0670 - accuracy: 0.9749 - val_loss: 1.1141 - val_accuracy: 0.7677
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0539 - accuracy: 0.9772 - val_loss: 0.9859 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0445 - accuracy: 0.9794 - val_loss: 1.4316 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0412 - accuracy: 0.9815 - val_loss: 1.3530 - val_accuracy: 0.7638


In [66]:
# Make predictions
model_4_pred_probs = model_4.predict(val_sentences)

24/24 [==============================] - 1s 3ms/step


In [67]:
# convert pred probs to pre labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))

In [68]:
# Calculate the results of model_4
model_4_results = calculate_results(y_true=val_labels,
                                    y_preds=model_4_preds)
model_4_results

{'accuracy': 76.37795275590551,
 'precision': 0.7635318416248198,
 'recall': 0.7637795275590551,
 'f1': 0.7628883537445144}

## CNN for text and other types of sequences

CNN's are typically for images, images are 2D. Text is 1D, `Conv1D` is what I'll be using

```
Inputs (text) -> Tokenization -> Embedding -> Layer(s) (typically Conv1D + pooling) -> Outputs
```

### Model 5: Conv1D

In [69]:
# Test out embedding layer, Conv1D layer and max pooling layer
embedding_test = embedding(text_vectorizer(["this is a test sentence"])) # turn target sequence into embedding
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation="relu",
                        padding="valid")

conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output) # get the most important feature / get feature with the highest value

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [70]:
# build conv1d model
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64, kernel_size=5, activation="relu", padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

# compile
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# summary of model
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)              

In [71]:
# fit
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20230125-145902
Epoch 1/5
215/215 [==============================] - 14s 6ms/step - loss: 0.1204 - accuracy: 0.9637 - val_loss: 0.8890 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0743 - accuracy: 0.9733 - val_loss: 1.0518 - val_accuracy: 0.7651
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0610 - accuracy: 0.9777 - val_loss: 1.0920 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0570 - accuracy: 0.9759 - val_loss: 1.1485 - val_accuracy: 0.7572
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0523 - accuracy: 0.9788 - val_loss: 1.1715 - val_accuracy: 0.7612


In [72]:
# make some predictions
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[4.0851688e-01],
       [8.3023393e-01],
       [9.9991596e-01],
       [7.1045786e-02],
       [4.4722270e-07],
       [9.9278581e-01],
       [9.3197459e-01],
       [9.9998200e-01],
       [9.9999857e-01],
       [7.9919451e-01]], dtype=float32)

In [73]:
# convert model 5 pred probs to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [74]:
# evaluate model 5
model_5_results = calculate_results(y_true=val_labels,
                                    y_preds=model_5_preds)
model_5_results

{'accuracy': 76.11548556430446,
 'precision': 0.761395918264994,
 'recall': 0.7611548556430446,
 'f1': 0.7597317731418467}

In [75]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 6: TensorFlow Hub Pretrained Sentence Encoder

In [ ]:
%conda install tensorflow-hub

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::flake8==4.0.1=pyhd8ed1ab_2
  - conda-forge/noarch::jupyter_client==7.3.4=pyhd8ed1ab_0
  - conda-forge/noarch::nltk==3.6.7=pyhd8ed1ab_0
  - conda-forge/linux-64::astropy==5.1=py310hde88566_0
  - conda-forge/noarch::flask==2.2.2=pyhd8ed1ab_0
  - conda-forge/noarch::nbformat==5.7.0=pyhd8ed1ab_0
  - conda-forge/noarch::flask-cors==3.0.10=pyhd8ed1ab_0
  - conda-forge/noarch::nbclient==0.7.0=pyhd8ed1ab_0
  - conda-forge/noarch::python-lsp-server==1.3.3=pyhd8ed1ab_0
  - conda-forge/linux-64::shap==0.41.0=py310h769672d_0
  - conda-forge/noarch::distributed==2022.10.0=pyhd8ed1ab_2
  - conda-forge/noarch::ipykernel==6.16.2=pyh210e3f2_0
  - conda-forge/noarch::nbconvert-core==7.2.3=pyhd8ed1ab_0
  - conda-forge/noarch::pyls-spyder==0.4.0=pyhd8ed1ab_0
  - conda-forge/noarch::python-lsp-black==1.1.0=pyhd8ed1ab_0
  - conda-forg

In [77]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence,
                      "When you call universal sentence encoder on a sentence, it converts it to a number"])

print(embed_samples[0][:50])

ModuleNotFoundError: No module named 'tensorflow_hub'